In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd 
from glob import glob 
from pathlib import Path 
import os 
from fastcore.xtras import load_pickle

import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams.update(mpl.rcParamsDefault)
plt.style.use(["science", "nature"])

import matplotx

In [3]:
all_out = glob('out/**/*.pkl')

In [4]:
len(all_out)

3

In [5]:
all_res = [load_pickle(p) for p in all_out]

In [7]:
all_res[-1].keys()

dict_keys(['baseline', 'accuracy', 'acc_macro', 'racc', 'kappa', 'confusion_matrix', 'f1_macro', 'f1_micro', 'frac_valid', 'all_y_true', 'all_y_pred', 'valid_indices', 'might_have_rounded_floats', 'train_size'])

In [9]:
all_res[-1]['all_y_pred']

,composition,is_metal
1942,In1Sb0.5As0.5,0
2380,Li11Mn13O32,1
1807,HgPt,1
3794,SmCuSeO,0
3349,PrFeGe2,1
...,...,...
3391,Rb2Cd3(B4O7)4,0
2259,La2B4Rh5,1
761,CdCu2SnS4,0
3869,Sn5(BRh3)2,1
